In [1]:
import math
import warnings
import numpy as np
import pandas as pd
from erfa import ErfaWarning

from utils_misc import reduce_targets, compare_targets, show_unique_items
from utils_wsi_epoch_prop import wsi_J2016_prop
from utils_proper_motion import total_pm

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=ErfaWarning)

# Goal
For stars in the WSI:
1. How many primaries have mags and proper motions in the wds?
2. How many secondaries have mags and proper motions in the wds?
3. Propagate positions of stars to J2016 for Gaia query.

### A note on column names
I've renamed columns in both the wds and wsi to keep better track of which data point belongs to which catalog.

Columns in the wds always start with "wds". For example, the first few are "wds_id", "wds_dd", "wds_comp", and so on. Similarly, the wsi would be "wsi_id", "wsi_dd", "wsi_comp"...

In [2]:
# read in data
# replace empty coords with AB
wsi = pd.read_csv('data/wsi24.sb.csv').replace({'wds_comp':np.NaN},'AB')
wds = pd.read_csv('data/wds.summ.csv').replace({'wds_comp':np.NaN},'AB')

# flag remaining nans and empty mag measurements to be located ('.' in wds)
wds = wds.replace({'wds_notes':np.NaN}, '') # dont flag empty entries in wds notes
wds = wds.replace([np.NaN, '.'],'%')

# force column data types as needed
wsi = wsi.astype( {'wds_id':str, 'wds_comp':str} )
wds = wds.astype( {'wds_id':str, 'wds_comp':str, 
                   'wds_mag1':str, 'wds_mag2':str,
                   'wds_coord':str} )

In [3]:
# remove target with no precise coord
wsi = wsi.drop(wsi.loc[ wsi.wds_id == '22267+6011' ].index)
wds = wds.drop(wds.loc[ wds.wds_id == '22267+6011' ].index)

In [4]:
# filter wsi
wsi = wsi.drop( columns=['wsi_ra_deg', 'wsi_dec_deg'] ) # drop RA and Dec columns
# wsi = wsi[ wsi['wsi_filter'] == 'y' ] # mask off all filters except y
wsi = wsi.loc[ ~wsi['wsi_dm'].isin([0.0,7.5]) ] # mask off any entry with a dm of 0.0 or 7.5 (pipeline error)
wsi = wsi.reset_index(drop=True)

In [5]:
# reduce wds to targets found in wsi
wds = reduce_targets(wds, wsi)
print( 'are unique targs the same for wds & wsi? how many are there?', compare_targets(wsi, wds) )
print( 'wsi length', len(wsi) )
print( 'wds length', len(wds) )

are unique targs the same for wds & wsi? how many are there? (True, 2137)
wsi length 2877
wds length 3809


In [6]:
# search for direct comp match between wsi and wds entry
# we should find a direct match for each, flag it if we dont
comps = []

for i, wds_id in enumerate( wsi['wds_id'] ): # loop through wds_ids in our wsi catalog

    # define the component for the current entry
    component = wsi.wds_comp.iloc[i]
    
    # find the primary match
    match = wds.loc[ (wds['wds_id']==wds_id) & (wds['wds_comp']==component) ]

    # try to add the component of the match to the list
    try:
        comps.append( match.wds_comp.iloc[0] )
    except:
        comps.append( '%' )

# check for number of flagged comps
comps.count('%')

1

there's only one missing. we can index it then look up the target in the wsi and wds to see what going on

In [7]:
# get index for flagged components
flagged_index = comps.index('%')

# look at the problem entry in the wsi
wsi.iloc[ flagged_index ]

wds_id                           23228+2034
wds_dd                            TOK 704Ba
wds_comp                                 Ba
wsi_date                          2021.7018
wsi_filter                                y
wsi_sep                              0.7492
wsi_sep_e                            0.0664
wsi_pa                               186.34
wsi_pa_e                               3.65
wsi_dm                                  1.7
wsi_dm_e                             0.8753
wsi_dm_flag                             NaN
wsi_nav                                   3
wsi_avg                                   1
sb_id1         Gaia DR3 2825753374534824320
sb_id2                                  NaN
sb_flg1                                   .
sb_flg2                                   $
Name: 2726, dtype: object

In [8]:
# look up this target in the wds
wds.loc[wds.wds_id=='23228+2034']

,wds_id,wds_dd,wds_comp,wds_date_first,wds_date_last,wds_num_obs,wds_pa_first,wds_pa_last,wds_sep_first,wds_sep_last,wds_mag1,wds_mag2,wds_spt,wds_pm1_ra,wds_pm1_dec,wds_pm2_ra,wds_pm2_dec,wds_durch_num,wds_notes,wds_coord
3708,23228+2034,STF3007,AB,1829,2022,284,79,92,5.7,5.9,6.74,9.78,G2V+dK6,313.0,-12.0,315.0,-11.0,+19 5093,NO,23 22 48.67 +20 33 32.2
3709,23228+2034,STF3007,AC,1887,2017,23,323,305,74.3,103.8,6.74,10.86,G2V,315.0,-11.0,4.0,-7.0,%,NL U,23 22 48.67 +20 33 32.2
3710,23228+2034,TOK 704,"Ba,Bb",2015,2021,6,181,187,0.7,0.8,6.4,8.4,dK6,315.0,-11.0,%,%,%,NO R,23 22 49.08 +20 33 32.1


it looks like the component in the wsi got missnamed to just Ba

the wds entry with component Ba,Bb is the correct match to our problem target. we can jut manually fix this then rerun the comp check

In [9]:
# fix entry in wsi
wsi.iat[flagged_index, 2] = 'Ba,Bb'

In [10]:
# rerun component check
ids = []
comps = []
for i, wds_id in enumerate( wsi['wds_id'] ): # loop through wds_ids in our wsi catalog
    component = wsi.wds_comp.iloc[i]
    match = wds.loc[ (wds['wds_id']==wds_id) & (wds['wds_comp']==component) ]
    try:
        ids.append( match.wds_id.iloc[0] )
        comps.append( match.wds_comp.iloc[0] )
    except:
        ids.append( '%' )
        comps.append( '%' )

# check for flagged comps
comps.count('%')

0

everything looks good, just make sure the ids and comps pulled match (in order) the ids and comps in the wsi

In [11]:
print( 'ids match?', ids == list(wsi.wds_id) )
print( 'comps match?', comps == list(wsi.wds_comp) )

ids match? True
comps match? True


now we can loop through the wsi, (knowing we get a component match for each observation) and add data from the wds to the wsi.

every observation will have a precise coordinate for the primary. They also _should_ have a mag and pm for the primary. we'll see how many secondaries have mags and pms

In [12]:
primary_coords = []
primary_mags = []
secondary_mags = []
primary_pms_ra = []
primary_pms_dec = []
secondary_pms_ra = []
secondary_pms_dec = []
wds_notes = []

for i, wds_id in enumerate( wsi['wds_id'] ): # loop through wds_ids in our wsi catalog

    # define component for current entry
    component = wsi.wds_comp.iloc[i]

    # find match based on current wds id and component
    match = wds.loc[ (wds['wds_id']==wds_id) & (wds['wds_comp']==component) ]

    # add precise primary coordinate
    primary_coords.append( match.wds_coord.iloc[0] )

    # add primary mag
    primary_mags.append( match.wds_mag1.iloc[0] )

    # add secondary mag
    secondary_mags.append( match.wds_mag2.iloc[0] )

    # add primary pms
    primary_pms_ra.append( match.wds_pm1_ra.iloc[0]  )
    primary_pms_dec.append( match.wds_pm1_dec.iloc[0] )
    
    # add secondary pms
    secondary_pms_ra.append( match.wds_pm2_ra.iloc[0] )
    secondary_pms_dec.append( match.wds_pm2_dec.iloc[0] )

    # add notes
    wds_notes.append( match.wds_notes.iloc[0] )

# add the lists to the wsi dataframe
wsi['wds_coord1'] = primary_coords
wsi['wds_mag1'] = primary_mags
wsi['wds_mag2'] = secondary_mags
wsi['wds_pm1_ra'] = primary_pms_ra
wsi['wds_pm1_dec'] = primary_pms_dec
wsi['wds_pm2_ra'] = secondary_pms_ra
wsi['wds_pm2_dec'] = secondary_pms_dec
wsi['wds_notes'] = wds_notes

In [13]:
# sanity check: compare the wds ids to the precise coordinates we just added
# the wds id should be a rounded version of the precise coords
print( wsi.wds_id.iloc[0], ':', wsi.wds_coord1.iloc[0] )
print( wsi.wds_id.iloc[-1], ':', wsi.wds_coord1.iloc[-1] )

18124-0718 : 18 12 24.02 -07 17 49.1
13203+6938 : 13 20 15.93 +69 38 08.3


now we can check for any missing mags and proper motions for the primaries and secondaries

In [14]:
# are we missing any primary mags?
list(wsi.wds_mag1).count('%')

0

all there. now we'll try secondaries

In [15]:
# are we missing any secondary mags?
list(wsi.wds_mag2).count('%')

1

we're missing one secondary mag. lets just get rid of that entry so it doesn't give us issues later

In [16]:
# drop row with missing secondary mag
wsi = wsi.drop( wsi.loc[ wsi['wds_mag2'] == '%' ].index ).reset_index(drop=True)

now lets check proper motions for the primary and secondary

In [17]:
# check primaries
print( list(wsi.wds_pm1_ra).count('%') )
print( list(wsi.wds_pm1_dec).count('%') )

2
2


In [18]:
wsi.loc[ wsi.wds_pm1_ra=='%' ]

,wds_id,wds_dd,wds_comp,wsi_date,wsi_filter,wsi_sep,wsi_sep_e,wsi_pa,wsi_pa_e,wsi_dm,...,sb_flg1,sb_flg2,wds_coord1,wds_mag1,wds_mag2,wds_pm1_ra,wds_pm1_dec,wds_pm2_ra,wds_pm2_dec,wds_notes
1230,02478+4700,J 882AB,AB,2020.8831,C,4.4171,0.0164,29.11,0.21,0.7,...,.,.,02 47 49.08 +47 01 18.6,9.8,9.8,%,%,56.0,-25.0,
2632,11563+3527,STT 241,AB,2021.2446,y,1.7945,0.0023,147.56,0.07,2.2,...,.,.,11 56 17.24 +35 26 53.3,6.82,8.74,%,%,-85.0,-23.0,O


it's odd that we have a pm for the secondary but not the primary. I'll just drop the target

In [19]:
# drop primary with no pm data
wsi = wsi.drop( wsi.loc[ wsi['wds_pm1_ra'] == '%' ].index ).reset_index(drop=True)

In [20]:
# check secondaries
# ra and dec counts should match
print( list(wsi.wds_pm2_ra).count('%') )
print( list(wsi.wds_pm2_dec).count('%') )

483
483


In [21]:
# length of wsi
len(wsi)

2874

# Conclusion
For the 2874 wsi 2024 targets:
* All primaries have magnitudes.
* All but one secondaries have magnitudes.
* All but two primaries have proper motions.
* 483 secondaries do not have proper motions.
* 83% of wsi targets have mags and proper motions for primary and secondary stars.

This sample does __NOT__ have  targets with a small separation or magnitude > 3 filtered out, which we might need to do to query gaia.

In [22]:
# force mags into float
wsi = wsi.astype({'wds_mag1':float, 'wds_mag2':float})

# remove targets brighter than mag 3
wsi = wsi.loc[ wsi['wds_mag1'] > 3.0 ]
wsi = wsi.loc[ wsi['wds_mag2'] > 3.0 ]

len(wsi) # check new length

2868

# Proper motion and epoch propagation

In [23]:
# replace flags
wsi = wsi.replace( {'wds_pm2_ra':'%', 'wds_pm2_dec':'%'}, np.nan )

# force data types
wsi = wsi.astype({'wds_pm1_ra':float, 'wds_pm1_dec':float,
                  'wds_pm2_ra':float, 'wds_pm2_dec':float}).reset_index(drop=True)

In [24]:
flags = []

# if the secondary is missing proper motion data, substitute the primary's pm and flag it
for i in range( len(wsi) ):

    # is the sec pm data a nan?
    if math.isnan( wsi.wds_pm2_ra.iloc[i] ):

        # substitute primary pm
        wsi.at[i, 'wds_pm2_ra']  = wsi.wds_pm1_ra.iloc[i]
        wsi.at[i, 'wds_pm2_dec'] = wsi.wds_pm1_dec.iloc[i]
        flags.append('!')
    
    # flag that is has its own pm
    else:
        flags.append('.')

wsi['epoch_prop_flag'] = flags

of the secondaries that do not have their own proper motion, are any flagged as not physically associated?

In [25]:
show_unique_items( wsi.loc[wsi.epoch_prop_flag=='!'].wds_notes )

 : 225
L : 1
N : 53
N  B : 1
N W : 1
NC : 3
NL : 2
NL U : 1
NO : 131
NO P : 2
NO R : 2
NOW : 1
NU : 3
NV : 7
NV U : 1
O : 39
O P : 2
T : 1
V : 3
W : 4


Before calculating total proper motion and propagating the positions, we need to correct the proper motions for the truncated values in the wds.

In [26]:
# which indexes have a P flag?
P_flag = list( wsi['wds_notes'].str.contains('P') )

In [27]:
# correct these proper motions
for i in range( len(wsi) ):
    if P_flag[i] == True:
        wsi.at[i, 'wds_pm1_ra']  = wsi.wds_pm1_ra.iloc[i] * 10
        wsi.at[i, 'wds_pm1_dec'] = wsi.wds_pm1_dec.iloc[i] * 10
        wsi.at[i, 'wds_pm2_ra']  = wsi.wds_pm2_ra.iloc[i] * 10
        wsi.at[i, 'wds_pm2_dec'] = wsi.wds_pm2_dec.iloc[i] * 10

In [28]:
# calculate total proper motions and add to wsi
wsi['wds_pm1'] = total_pm( wsi.wds_pm1_ra, wsi.wds_pm1_dec )
wsi['wds_pm2'] = total_pm( wsi.wds_pm2_ra, wsi.wds_pm2_dec )

In [29]:
# propogate proper motions
wsi = wsi_J2016_prop( wsi )

In [30]:
# force data types
wsi = wsi.astype({'wds_mag1':float, 'wds_mag2':float,
                  'wds_pm1':float, 'wds_pm2':float}).reset_index(drop=True)

wsi.to_csv('data/wsi24.sb.prop.csv', index=False)

In [31]:
wsi

,wds_id,wds_dd,wds_comp,wsi_date,wsi_filter,wsi_sep,wsi_sep_e,wsi_pa,wsi_pa_e,wsi_dm,...,wds_pm1,wds_pm2,wds_ra1_J2000,wds_dec1_J2000,wds_ra2_J2000,wds_dec2_J2000,wds_ra1_J2016,wds_dec1_J2016,wds_ra2_J2016,wds_dec2_J2016
0,18124-0718,"A 36A,BC","A,BC",2020.5356,V,71.2909,0.0207,39.00,0.02,5.6,...,54.626001,15.264338,273.100083,-7.296972,273.112647,-7.281582,273.099985,-7.297194,273.112589,-7.281547
1,18124-0718,"A 36A,BC","A,BC",2020.5356,y,71.1704,0.0264,38.95,0.02,5.5,...,54.626001,15.264338,273.100083,-7.296972,273.112612,-7.281597,273.099985,-7.297194,273.112554,-7.281562
2,05192-0304,A 53AB,AB,2022.0851,y,1.6830,0.0164,294.29,0.58,3.4,...,713.473195,660.461960,79.802750,-3.073806,79.802323,-3.073613,79.805866,-3.073192,79.805230,-3.073173
3,06486-0405,A 58AB,AB,2021.1702,y,5.0778,0.0023,161.43,0.03,1.3,...,94.667840,94.667840,102.149083,-4.075722,102.149534,-4.077059,102.148865,-4.076082,102.149315,-4.077419
4,05424+1259,A 116,AB,2022.1066,y,1.6954,0.0136,291.63,0.46,3.2,...,10.630146,10.630146,85.593958,12.977944,85.593509,12.978118,85.593922,12.977913,85.593473,12.978087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,08364+6718,YR 13,AB,2022.2191,y,0.5926,0.0530,298.81,3.83,1.8,...,1082.266141,1082.266141,129.106042,67.295111,129.105668,67.295190,129.093606,67.295422,129.093232,67.295501
2864,03496-0220,YR 23,AB,2022.0846,y,0.3433,0.0025,283.47,0.54,0.6,...,74.330344,74.330344,57.411167,-2.328472,57.411074,-2.328450,57.410891,-2.328654,57.410798,-2.328632
2865,10183+0841,YSC 93,AB,2021.3046,V,0.3299,0.0049,104.96,0.86,1.5,...,6.082763,6.082763,154.579875,8.682250,154.579965,8.682226,154.579879,8.682223,154.579969,8.682200
2866,10183+0841,YSC 93,AB,2021.3046,y,0.3972,0.0080,93.05,1.16,1.6,...,6.082763,6.082763,154.579875,8.682250,154.579986,8.682244,154.579879,8.682223,154.579991,8.682217
